In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/chriskyfung/pandas_stock_notebook.git
%cd pandas_stock_notebook
!ls

In [ ]:
!pip install -r requirements.txt

In [2]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
from datetime import date, timedelta
import pandas as pd
import pandas_market_calendars as mcal

# Create a calendar
hkex = mcal.get_calendar('HKEX')

today = date.today()
past7days = today - timedelta(days=7)
print("Today's date:", today)

# Show available calendars in the past 7 days
open_dates = hkex.valid_days(past7days,today)
print(f'\nOpen dates (past 7 days):\n {open_dates.date}')

# Show the last stock open date
last_open_date = open_dates[-1].date()
print(f'\nLast open date: {last_open_date}')

Today's date: 2021-07-04

Open dates (past 7 days):
 [datetime.date(2021, 6, 28) datetime.date(2021, 6, 29)
 datetime.date(2021, 6, 30) datetime.date(2021, 7, 2)]

Last open date: 2021-07-02


In [4]:
from stocks import stock_profile # 從 stocks.py 載入 stock_profile 類別
  
stocks = stock_profile()
stocks.loadJsonProfile('./stocks.json') # 從JSON文件讀入股票組合清單及股票名稱索引
tickers = list(stocks.listing.keys())
stocks.cacheFromQuandl(tickers, last_open_date) # 從Yahoo下載股票組合之最近一天資料


港交所(0388.hk):


,NominalPrice,NetChange,Change,Bid,Ask,PEx,High,Low,PreviousClose,ShareVolume000,Turnover000,LotSize
Date,,,,,,,,,,,,
2021-07-02,455.8,NaN,NaN,455.6,455.8,NaN,463.8,454.4,462.8,5813.0,2653112.0,NaN




騰訊(0700.hk):


,NominalPrice,NetChange,Change,Bid,Ask,PEx,High,Low,PreviousClose,ShareVolume000,Turnover000,LotSize
Date,,,,,,,,,,,,
2021-07-02,574.5,NaN,NaN,574.0,574.5,NaN,598.5,572.5,584.0,24939.0,14455214.0,NaN




中石油(0857.hk):


...

,NominalPrice,NetChange,Change,Bid,Ask,PEx,High,Low,PreviousClose,ShareVolume000,Turnover000,LotSize
Date,,,,,,,,,,,,
2021-07-02,221.2,NaN,NaN,221.0,221.2,NaN,226.0,216.0,232.2,12138.0,2684327.0,NaN




小米(1810.hk):


,NominalPrice,NetChange,Change,Bid,Ask,PEx,High,Low,PreviousClose,ShareVolume000,Turnover000,LotSize
Date,,,,,,,,,,,,
2021-07-02,26.2,NaN,NaN,26.15,26.2,NaN,27.0,25.95,27.0,134959.0,3530458.0,NaN




舜宇光學(2382.hk):


,NominalPrice,NetChange,Change,Bid,Ask,PEx,High,Low,PreviousClose,ShareVolume000,Turnover000,LotSize
Date,,,,,,,,,,,,
2021-07-02,236.6,NaN,NaN,236.6,236.8,NaN,244.8,230.0,245.4,7451.0,1754328.0,NaN


In [6]:
from termcolor import colored
import re

asked_stock = input('Enter a stock code/name: ') # 詢問要查閱的股票代號/名稱

default_suffix = '.hk'
stock_code = None
stock_name = None

# 判辨輸入的字串是股票代號還是股票名稱
match = re.match('^(?P<code>\d+)(?P<suffix>\.\w+)?$', asked_stock)
if match:
  stock_code = asked_stock
  if match.group('suffix') == None:
    stock_code = stock_code + default_suffix
  stock_code = stock_code.lower().zfill(7)
  stock_name = stocks.code2name(stock_code)
else:
  stock_code = stocks.name2code(asked_stock)
  stock_name = asked_stock
# 從快取要查閱的股票資料
if (stock_code in stocks.dataframes):
  df = stocks.dataframes[stock_code]
  columns = list(df) # or df.columns
  # 將最近一天的資料Print到屏幕輸出
  print(f'\nStock data of {stock_name}({stock_code}) on the last open day ({last_open_date})\n')
  for column in columns:
    print(f'{column:14s}: {df.iloc[-1].loc[column]:>15,.2f}')
else:
  # 如果沒有快取，則輸出紅色的文字提示！
  print(colored(f'\nCannot find "{asked_stock}" from the cached data', 'red'))


Stock data of 小米(1810.hk) on the last open day (2021-07-02)

NominalPrice  :           26.20
NetChange     :             nan
Change        :             nan
Bid           :           26.15
Ask           :           26.20
PEx           :             nan
High          :           27.00
Low           :           25.95
PreviousClose :           27.00
ShareVolume000:      134,959.00
Turnover000   :    3,530,458.00
LotSize       :             nan


In [ ]:
import yfinance as yf

ticker = yf.Ticker("0001.hk")
ticker.history(period='1day')